# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")

In [4]:
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)


def compare(baselines_dict, da_or_ds, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        if "dimname" in baselines_dict:
            print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        else:
            print(f"|  | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if "dimname" in baselines_dict:
        all_vars = da_or_ds[baselines_dict["dimname"]].data
    else:
        all_vars = baselines_dict["values"]
    for var in all_vars:
        if "dimname" in baselines_dict:
            da = da_or_ds
            sel_dict = {baselines_dict["dimname"]: var}
        else:
            da = da_or_ds[var]
            sel_dict = {}
        matlab_val = baselines_dict["values"][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(
                f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}"
                + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
            )

In [5]:
compare(
    baselines["day1"]["forcing"],
    testcase.forcing.isel(time=0, X=0, zooplankton=0),
    markdown_formatting=True,
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| T_bottom | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| zooC | 3.3287e+00 | 3.3287e+00 | 0.0000e+00 |
| poc_flux_bottom | 2.7847e-02 | 2.7847e-02 | 1.2459e-16 |
| zoo_mort | 7.7564e-01 | 7.7564e-01 | 0.0000e+00 |


## Run the model


In [6]:
%%time

nsteps = 365
make_plot = False
testcase.run(nsteps)
testcase.ds

CPU times: user 1min 21s, sys: 26.5 ms, total: 1min 21s
Wall time: 1min 23s


<xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 365, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.999e-06 ... 0.05344
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 2.234e-05
    predation_flux             (time, fish, X) float64 5.737e-12 ... 0.0
    predation_rate             (time, fish, X) float64 5.737e-07 ... 0.0
    metabolism_rate            (time, fish, X) float64 0.04296 ... 0.00488
    ...                         ...
    recruitment_flux           (time, fish, X) float64 1.453e-09 ... 0.0
    fish_catch_rate            (time, fish, X) float64 0.0 0.0 ... 0.0008219
    encounter_rate_link        (time, feeding_link, X) float64 2.288 ... 0.0
    encounter_rate_total       (time, fish, X) float64 2.288 2.288 ... 2.239e-05
    consumption_rate_max_pred  (time, fish, X) float64 0.2375 0.2375 ... 0.01062
    consumption_rate_link      (time, feeding_link, X) float64 0.2152 ... 0.0

In [7]:
if nsteps > 1:
    check_key = f"day{nsteps}"
    test_ds = testcase.ds.isel(X=0, time=-1)
    if make_plot:
        for group in test_ds.group.data:
            testcase.ds.biomass.sel(group=group).isel(X=0).plot()
        plt.ylim([1e-6, 1])
        plt.yscale("log")
        plt.legend(test_ds.group.data)
else:
    check_key = "day1"
    test_ds = testcase.ds.isel(X=0, time=0)

In [8]:
compare(
    baselines[check_key]["T_habitat"],
    test_ds["T_habitat"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Sp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Sd | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Mf | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Mp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Md | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| Lp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Ld | 4.2086e+00 | 4.2086e+00 | 0.0000e+00 |


In [9]:
compare(
    baselines[check_key]["metabolism"],
    test_ds["metabolism_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Sp | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Sd | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Mf | 1.4479e-02 | 1.4479e-02 | 2.3961e-16 |
| Mp | 1.4479e-02 | 1.4479e-02 | 2.3961e-16 |
| Md | 4.3744e-03 | 4.3744e-03 | 0.0000e+00 |
| Lp | 4.8801e-03 | 4.8801e-03 | 0.0000e+00 |
| Ld | 1.4754e-03 | 1.4754e-03 | 2.9393e-16 |


In [10]:
compare(
    baselines[check_key]["encounter"],
    test_ds["encounter_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.2833e+00 | 2.2833e+00 | 1.9449e-16 |
| Sp_Zoo | 2.2833e+00 | 2.2833e+00 | 1.9449e-16 |
| Sd_Zoo | 2.2833e+00 | 2.2833e+00 | 1.9449e-16 |
| Mf_Zoo | 2.9647e-01 | 2.9647e-01 | 5.6172e-16 |
| Mf_Sf | 3.4993e-06 | 3.4993e-06 | 2.4206e-16 |
| Mf_Sp | 1.8581e-06 | 1.8581e-06 | 3.4190e-16 |
| Mf_Sd | 1.8743e-06 | 1.8743e-06 | 3.3894e-16 |
| Mp_Zoo | 2.9647e-01 | 2.9647e-01 | 5.6172e-16 |
| Mp_Sf | 3.4993e-06 | 3.4993e-06 | 2.4206e-16 |
| Mp_Sp | 1.8581e-06 | 1.8581e-06 | 3.4190e-16 |
| Mp_Sd | 1.8743e-06 | 1.8743e-06 | 3.3894e-16 |
| Md_benthic_prey | 5.2100e-02 | 5.2100e-02 | 1.3319e-16 |
| Lp_Mf | 1.1638e-05 | 1.1638e-05 | 4.3669e-16 |
| Lp_Mp | 1.8216e-05 | 1.8216e-05 | 0.0000e+00 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 3.6153e-06 | 3.6153e-06 | 1.1715e-16 |
| Ld_Mp | 5.6586e-06 | 5.6586e-06 | 1.4969e-16 |
| Ld_Md | 7.5040e-06 | 7.5040e-06 | 0.0000e+00 |
| Ld_benthic_prey | 

In [11]:
compare(
    baselines[check_key]["consumption"],
    test_ds["consumption_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Sp_Zoo | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Sd_Zoo | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Mf_Zoo | 4.2952e-02 | 4.2952e-02 | 1.6155e-16 |
| Mf_Sf | 5.0697e-07 | 5.0697e-07 | 2.0885e-16 |
| Mf_Sp | 2.6920e-07 | 2.6920e-07 | 1.9666e-16 |
| Mf_Sd | 2.7154e-07 | 2.7154e-07 | 1.9496e-16 |
| Mp_Zoo | 4.2952e-02 | 4.2952e-02 | 1.6155e-16 |
| Mp_Sf | 5.0697e-07 | 5.0697e-07 | 2.0885e-16 |
| Mp_Sp | 2.6920e-07 | 2.6920e-07 | 1.9666e-16 |
| Mp_Sd | 2.7154e-07 | 2.7154e-07 | 1.9496e-16 |
| Md_benthic_prey | 1.4862e-02 | 1.4862e-02 | 2.3344e-16 |
| Lp_Mf | 1.1605e-05 | 1.1605e-05 | 1.4597e-16 |
| Lp_Mp | 1.8164e-05 | 1.8164e-05 | 3.7305e-16 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 8.1749e-07 | 8.1749e-07 | 0.0000e+00 |
| Ld_Mp | 1.2795e-06 | 1.2795e-06 | 1.6550e-16 |
| Ld_Md | 1.6968e-06 | 1.6968e-06 | 2.4960e-16 |
| Ld_benthic_prey | 

In [12]:
compare(
    baselines[check_key]["ingestion"],
    test_ds["ingestion_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Sp | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Sd | 2.1514e-01 | 2.1514e-01 | 0.0000e+00 |
| Mf | 4.2953e-02 | 4.2953e-02 | 1.6155e-16 |
| Mp | 4.2953e-02 | 4.2953e-02 | 1.6155e-16 |
| Md | 1.4862e-02 | 1.4862e-02 | 2.3344e-16 |
| Lp | 2.9770e-05 | 2.9770e-05 | 2.2762e-16 |
| Ld | 3.4049e-03 | 3.4049e-03 | 0.0000e+00 |


In [13]:
compare(
    baselines[check_key]["predation"],
    test_ds["predation_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.0833e-05 | 2.0833e-05 | 1.6263e-16 |
| Sp | 2.0833e-05 | 2.0833e-05 | 1.6263e-16 |
| Sd | 2.0833e-05 | 2.0833e-05 | 1.6263e-16 |
| Mf | 1.3917e-05 | 1.3917e-05 | 6.0864e-16 |
| Mp | 2.7834e-05 | 2.7834e-05 | 7.3036e-16 |
| Md | 1.2065e-06 | 1.2065e-06 | 5.2656e-16 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [14]:
compare(
    baselines[check_key]["mortality"],
    test_ds["mortality_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sd | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Md | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Lp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Ld | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |


In [15]:
compare(
    baselines[check_key]["fishing"],
    test_ds["fish_catch_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 8.2192e-04 | 8.2192e-04 | 0.0000e+00 |
| Mp | 8.2192e-05 | 8.2192e-05 | 1.6489e-16 |
| Md | 8.2192e-05 | 8.2192e-05 | 1.6489e-16 |
| Lp | 8.2192e-04 | 8.2192e-04 | 0.0000e+00 |
| Ld | 8.2192e-04 | 8.2192e-04 | 0.0000e+00 |


In [16]:
compare(
    baselines[check_key]["avail_energy"],
    test_ds["energy_avail_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0764e-01 | 1.0764e-01 | 3.8679e-16 |
| Sp | 1.0764e-01 | 1.0764e-01 | 3.8679e-16 |
| Sd | 1.0764e-01 | 1.0764e-01 | 3.8679e-16 |
| Mf | 1.5588e-02 | 1.5588e-02 | 6.6773e-16 |
| Mp | 1.5588e-02 | 1.5588e-02 | 6.6773e-16 |
| Md | 6.0290e-03 | 6.0290e-03 | 2.8773e-16 |
| Lp | -4.8593e-03 | -4.8593e-03 | -0.0000e+00 |
| Ld | 9.0796e-04 | 9.0796e-04 | 0.0000e+00 |


In [17]:
compare(
    baselines[check_key]["growth"],
    test_ds["growth_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0756e-01 | 1.0756e-01 | 3.8706e-16 |
| Sp | 1.0756e-01 | 1.0756e-01 | 3.8706e-16 |
| Sd | 1.0756e-01 | 1.0756e-01 | 3.8706e-16 |
| Mf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mp | 1.5239e-02 | 1.5239e-02 | 3.4150e-16 |
| Md | 5.6881e-03 | 5.6881e-03 | 3.0497e-16 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [18]:
compare(
    baselines[check_key]["reproduction"],
    test_ds["reproduction_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.4510e-02 | 1.4510e-02 | 4.7820e-16 |
| Mp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 4.5408e-04 | 4.5408e-04 | 1.1938e-16 |


In [19]:
compare(
    baselines[check_key]["recruitment"],
    test_ds["recruitment_flux"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 5.9010e-08 | 5.9010e-08 | 3.3642e-16 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 1.0220e-09 | 1.0220e-09 | 0.0000e+00 |
| Mf | 1.8975e-06 | 1.8975e-06 | 4.4638e-16 |
| Mp | 1.0076e-06 | 1.0076e-06 | 2.1017e-16 |
| Md | 1.0164e-06 | 1.0164e-06 | 4.1669e-16 |
| Lp | 4.8500e-06 | 4.8500e-06 | 3.4929e-16 |
| Ld | 1.8005e-06 | 1.8005e-06 | 5.8805e-16 |


In [20]:
compare(
    baselines[check_key]["biomass"],
    test_ds["biomass"],
    markdown_formatting=True,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.7696e-05 | 1.7696e-05 | 3.8292e-16 |
| Sp | 9.3653e-06 | 9.3653e-06 | 0.0000e+00 |
| Sd | 9.4480e-06 | 9.4480e-06 | 0.0000e+00 |
| Mf | 4.0856e-04 | 4.0856e-04 | 1.3269e-16 |
| Mp | 3.1926e-04 | 3.1926e-04 | 3.3960e-16 |
| Md | 3.1755e-04 | 3.1755e-04 | 1.7071e-16 |
| Lp | 4.7386e-04 | 4.7386e-04 | 4.5760e-16 |
| Ld | 2.2672e-04 | 2.2672e-04 | 1.1955e-16 |
| benthic_prey | 6.3447e-01 | 6.3447e-01 | 0.0000e+00 |
